In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install marlin-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 16.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinu

In [ ]:
from marlin_pytorch import Marlin
# Load MARLIN model from GitHub Release
model = Marlin.from_online("marlin_vit_base_ytf")

In [ ]:
model

Marlin(
  (encoder): MarlinEncoder(
    (patch_embedding): PatchEmbedding3d(
      (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      (rearrange): Rearrange('b d nt nh nw -> b (nt nh nw) d')
    )
    (pos_embedding): SinCosPositionalEmbedding()
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (layers): ModuleList(
            (0): Linear(
              (linear): Linear(in_features=768, out_features=3072, bias=True)
              (activatio

In [ ]:
features = model.extract_video("/content/video.mp4", crop_face=True)
print(features.shape)

Start to analyze the face detection model, model path: .marlin/FaceXZoo/models, model category: face_detection，model name: face_detection_1.0
Successfully parsed the model configuration file meta.json!
Successfully loaded the face detection model!


/usr/local/lib/python3.10/dist-packages/torch/serialization.py:1101: SourceChangeWarning: source code of class 'torchvision.models._utils.IntermediateLayerGetter' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:1101: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:1101: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's s

torch.Size([4, 768])


In [ ]:
features = model.extract_video("/content/dia1375utt1.mp4", crop_face=True)
print(features.shape)

torch.Size([3, 768])


In [ ]:
features = model.extract_video("/content/dia1375utt2.mp4", crop_face=True)
print(features.shape)

torch.Size([1, 768])


In [ ]:
from google.colab import drive
from marlin_pytorch import Marlin
import os
import numpy as np
import glob
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

# # This will prompt for authorization.
# drive.mount('/content/drive')

# Directory containing your video files
video_directory = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/video_with_audio/train'

# Specify the target location where you want to save the features.
# Ensure this path exists in your Google Drive or adjust accordingly.
target_location = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train'


# Load MARLIN model from GitHub Release
model = Marlin.from_online("marlin_vit_base_ytf")

# List all mp4 files in the directory
video_files = glob.glob(os.path.join(video_directory, '*.mp4'))

skipped = 0

# Iterate over each video file in the directory with a tqdm progress bar
for video_path in tqdm(video_files, desc='Processing videos'):
    # Get the base name of the video file without extension
    video_filename = os.path.basename(video_path)
    filename_without_extension = os.path.splitext(video_filename)[0]

    # Construct the full path for saving the features
    save_path = os.path.join(target_location, filename_without_extension + '.npy')

    # Check if the embedding file already exists
    if os.path.exists(save_path):
        # print(f"Embedding for {video_filename} already exists. Skipping...")
        skipped += 1
        continue

    # Extract features since the embedding file does not exist
    features = model.extract_video(video_path, crop_face=True)
    # print(f"Processed {video_filename}, feature shape: {features.shape}")

    # Convert torch tensor to numpy array
    features_np = features.detach().cpu().numpy()
    # Compute mean along axis 0
    features_mean = np.mean(features_np, axis=0)

    # Save the features
    np.save(save_path, features_mean)
    # print(f"Features saved to {save_path}")
print("Total Skipped:", skipped)

Processing videos:   0%|          | 0/9976 [00:00<?, ?it/s]

Start to analyze the face detection model, model path: .marlin/FaceXZoo/models, model category: face_detection，model name: face_detection_1.0
Successfully parsed the model configuration file meta.json!
Successfully loaded the face detection model!


KeyboardInterrupt: 

In [ ]:
import os
import csv

video_directory = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/video_with_audio/train'
target_location = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train'


# Get filenames without extension from video_directory
video_files = [os.path.splitext(f)[0] for f in os.listdir(video_directory) if os.path.isfile(os.path.join(video_directory, f))]

# Get filenames without extension from target_location
target_files = [os.path.splitext(f)[0] for f in os.listdir(target_location) if os.path.isfile(os.path.join(target_location, f))]

# Find files in video_directory that don't exist in target_location
missing_files = [f for f in video_files if f not in target_files]

# Split the missing_files list into 6 equal parts
num_parts = 6
chunk_size = len(missing_files) // num_parts
missing_files_chunks = [missing_files[i:i + chunk_size] for i in range(0, len(missing_files), chunk_size)]

# Write each part to a separate CSV
for i, chunk in enumerate(missing_files_chunks):
    csv_file = f'missing_files_part_{i+1}.csv'
    if i != 0:
      # Remove the first occurrence of 'CSCI535/' from video_directory path
      video_directory = video_directory.replace('CSCI535/', '', 1)

    with open(csv_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'video_path']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for filename in chunk:
            video_path = os.path.join(video_directory, filename + ".mp4")
            writer.writerow({'filename': filename, 'video_path': video_path})

print("CSV files with missing files split into 6 parts successfully.")


CSV files with missing files split into 6 parts successfully.


In [ ]:
from google.colab import drive
from marlin_pytorch import Marlin
import os
import numpy as np
import csv
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

# # This will prompt for authorization.
# drive.mount('/content/drive')

# Specify the target location where you want to save the features.
# Ensure this path exists in your Google Drive or adjust accordingly.
target_location = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train/'

# Load MARLIN model from GitHub Release
model = Marlin.from_online("marlin_vit_base_ytf")

# Function to process videos and save features
def process_videos(video_files, target_location):
    skipped = 0

    # Iterate over each video file in the list with a tqdm progress bar
    for video_path in tqdm(video_files, desc='Processing videos'):
      try:
        # Get the base name of the video file without extension
        video_filename = os.path.basename(video_path)
        filename_without_extension = os.path.splitext(video_filename)[0]

        # Construct the full path for saving the features
        save_path = os.path.join(target_location, filename_without_extension + '.npy')

        # Check if the embedding file already exists
        if os.path.exists(save_path):
            # print(f"Embedding for {video_filename} already exists. Skipping...")
            skipped += 1
            continue
        # print(video_path)
        # Extract features since the embedding file does not exist
        features = model.extract_video(video_path, crop_face=True)
        # Convert torch tensor to numpy array
        features_np = features.detach().cpu().numpy()
        # Compute mean along axis 0
        features_mean = np.mean(features_np, axis=0)
        # print(f"Processed {video_filename}, feature shape: {features.shape}")

        # Save the features
        np.save(save_path, features_mean)
        # print(f"Features saved to {save_path}")
      except:
        print(video_path)
        break

    print("Total Skipped:", skipped)

# Select one of the 6 CSV files containing video file paths
csv_file_path = '/content/missing_files_part_1.csv'

# Read video file paths from the selected CSV file
video_files = []
with open(csv_file_path, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    for row in csvreader:
        video_files.append(row[1])

# Process the videos and save features
process_videos(video_files, target_location)


Processing videos:   0%|          | 0/143 [00:00<?, ?it/s]

Total Skipped: 1


In [ ]:
import os
import csv
import glob

# Directory containing your video files
video_directory = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/video_with_audio/test'

# List all mp4 files in the directory
video_files = glob.glob(os.path.join(video_directory, '*.mp4'))

# Calculate the number of files per partition
files_per_partition = len(video_files) // 6

# Create 6 partitions
partitions = [video_files[i:i+files_per_partition] for i in range(0, len(video_files), files_per_partition)]

# Function to remove the first occurrence of "CSCI535" from a file path
def remove_csci535(path):
    return path.replace('/CSCI535', '', 1)

# Write each partition to a separate CSV file
for i, partition in enumerate(partitions):
    with open(f'partition_{i+1}.csv', 'w+', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['File Path'])
        for file_path in partition:
            if i == 0:
              writer.writerow([file_path])
            else:
              file_path = remove_csci535(file_path)
              writer.writerow([file_path])


Mean Dim


In [ ]:
import numpy as np
import os
import glob
from tqdm import tqdm

# Define the source directory of your embeddings
source_location = '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/Video'

# Define the target directory for the averaged embeddings
new_target_location = '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/video_v2'

# Create the target directory if it doesn't exist
os.makedirs(new_target_location, exist_ok=True)

# List all .npy files in the source directory
embedding_files = glob.glob(os.path.join(source_location, '*.npy'))
skipped = 0
processed = 0

for embedding_file in tqdm(embedding_files, desc="Processing files"):
    # Construct the path for the new embedding file
    basename = os.path.basename(embedding_file)
    new_embedding_path = os.path.join(new_target_location, basename)

    # Check if the averaged embedding file already exists
    if os.path.exists(new_embedding_path):
        # print(f"File {basename} already processed. Skipping...")
        skipped += 1
        continue

    processed += 1
    # Load the embedding
    embedding = np.load(embedding_file)

    # Compute the mean along the first axis to get a shape of (1, 768)
    averaged_embedding = np.mean(embedding, axis=0, keepdims=True)

    # Save the averaged embedding
    np.save(new_embedding_path, averaged_embedding)
    # print(f"Processed and saved averaged embedding for {basename}")
print()
print("Skipped files", skipped)
print("Processed files", processed)

Processing files: 100%|██████████| 1087/1087 [00:00<00:00, 4449.17it/s]


Skipped files 1087
Processed files 0


# dim check

In [ ]:
import os
import numpy as np

# Directory path
directory = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/final_concat_data/train/'

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.npy'):
        # Load numpy array
        filepath = os.path.join(directory, filename)
        array = np.load(filepath)
        # print(array.shape)
        # break
        # Check shape
        if array.shape != (1, 2560):
            print(f"File {filename} has shape {array.shape}.")
        # else:
        #     print(f"File {filename} has correct shape (1, 768).")

print("Shape checking complete.")


# File check

In [ ]:
import os

# File path to check
file_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/datasets.zip'

# Check if file exists
if os.path.exists(file_path):
    print("File exists.")
else:
    print("File does not exist.")


File exists.


# Fiile Count

In [ ]:
import os

def count_files(directory):
    return len([file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))])

directory = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/'
file_count = count_files(directory)
print(f'There are {file_count} files in the directory.')

There are 11053 files in the directory.


In [ ]:
import os

def count_same_files(dir1, dir2):
    files_in_dir1 = set(os.listdir(dir1))
    files_in_dir2 = set(os.listdir(dir2))
    same_files = files_in_dir1 & files_in_dir2
    return len(same_files)

dir1 = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train/'
dir2 = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/'
num_same_files = count_same_files(dir1, dir2)
print(f'There are {num_same_files} files with the same name in both directories.')

There are 4158 files with the same name in both directories.


In [ ]:
import os
import shutil

dir1 = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train/'
dir2 = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/'

for filename in os.listdir(dir1):
    file_path = os.path.join(dir1, filename)
    if os.path.isfile(file_path):
        destination_path = os.path.join(dir2, filename)
        if not os.path.exists(destination_path):
            shutil.copy(file_path, dir2)
            # print(f"Copy {filename} to {dir2}")

In [ ]:
!rm '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia360utt19.npy'

In [ ]:
import os
import numpy as np
from tqdm import tqdm

def reshape_arrays(directory):
    # Loop through all files in the directory
    for filename in tqdm(os.listdir(directory), desc="Processing files"):
        # Check if the file is a .npy file
        if filename.endswith(".npy"):
            # Load the array from the file
            array = np.load(os.path.join(directory, filename))

            # Check if the array has the shape (768,)
            if array.shape == (768,):
                # Reshape the array in place to (1, 768)
                array = array.reshape((1, 768))

                # Save the reshaped array back to the file
                np.save(os.path.join(directory, filename), array)

# Call the function with the directory path
reshape_arrays('/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/')

Processing files: 100%|██████████| 11052/11052 [06:46<00:00, 27.19it/s] 


# Create Zip

In [ ]:
import os
import zipfile
from tqdm import tqdm

def zip_dirs(dir_paths, zip_path):
    """
    Zip all files in the given directories into a zip file.

    Args:
    - dir_paths (list): List of paths to the directories containing the files to be zipped.
    - zip_path (str): Path to the zip file to be created.
    """
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        files = []
        for dir_path in dir_paths:
            for root, _, filenames in os.walk(dir_path):
                for filename in filenames:
                    files.append(os.path.join(root, filename))
        with tqdm(total=len(files), desc='Zipping files', unit='file') as pbar:
            for file in files:
                zipf.write(file, os.path.relpath(file, os.path.commonprefix(dir_paths)))
                pbar.update(1)

# Directories to be zipped
dir_to_zip1 = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/'
dir_to_zip2 = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/test_reshaped/'

# Path for the zip file to be created
zip_file_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/datasets.zip'

# Zip the directories
zip_dirs([dir_to_zip1, dir_to_zip2], zip_file_path)

print("Directories zipped successfully!")

Zipping files: 100%|██████████| 13619/13619 [04:44<00:00, 47.89file/s] 


Directories zipped successfully!


In [ ]:
!zip -r datasets.zip /content/drive/MyDrive/CSCI535/CSCI535\ Project/Dataset/Processed/Video/train_reshaped /content/drive/MyDrive/CSCI535/CSCI535\ Project/Dataset/Processed/Video/test_reshaped

Streaming output truncated to the last 5000 lines.
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt11.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt12.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt13.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt14.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt15.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt16.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train_reshaped/dia1001utt17.npy (deflated 7%)
  adding: content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Pro

In [ ]:
import json
import csv

# Define the path to the JSON file
json_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/text/Subtask_2_train.json'

# Define the path for the output CSV file
csv_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/video_emotion_pairs.csv'

# Open the JSON file and load the data
with open(json_path, 'r') as json_file:
    data = json.load(json_file)

# Open the CSV file for writing
with open(csv_path, 'w+', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header row to the CSV file
    csv_writer.writerow(['video_name', 'emotion'])

    # Loop through each conversation in the JSON data
    for conversation in data:
        # Loop through each utterance in the conversation
        for utterance in conversation['conversation']:
            # Extract the video name and emotion
            video_name = utterance['video_name']
            emotion = utterance['emotion']

            # Write the video name and emotion to the CSV file
            csv_writer.writerow([video_name, emotion])

print('CSV file has been created with video name and emotion pairs.')


CSV file has been created with video name and emotion pairs.


In [ ]:
import os

# Define the source directory of your embeddings
source_location = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video'

# List all files in the source directory
all_files = os.listdir(source_location)

# Filter only the filenames (excluding directories)
file_names = [file for file in all_files if os.path.isfile(os.path.join(source_location, file))]

print("No of filenames:")
print(len(file_names))


No of filenames:
1087


In [ ]:
import csv
import os

# Define the source directory of your embeddings
source_location = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/video_v2'

# List all files in the source directory
file_names = [file for file in os.listdir(source_location) if os.path.isfile(os.path.join(source_location, file))]

print(len(file_names))


1087


In [ ]:
import csv

# Define the path for the existing CSV file
csv_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/video_emotion_pairs.csv'

# List of video names to keep
video_names_to_keep = set(file_names)  # Replace with your list

# Convert all video names to lowercase for case-insensitive comparison
video_names_to_keep_lower = [name.lower() for name in video_names_to_keep]

# Define the path for the new CSV file
new_csv_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/filtered_video_emotion_pairs.csv'

# Open the existing CSV file and create a new CSV file for writing
with open(csv_path, 'r', newline='') as existing_csv_file, open(new_csv_path, 'w', newline='') as new_csv_file:
    existing_csv_reader = csv.reader(existing_csv_file)
    new_csv_writer = csv.writer(new_csv_file)

    # Read and write the header row
    header = next(existing_csv_reader)
    new_csv_writer.writerow(header)

    # Find the index of the 'video_name' column
    video_name_index = header.index('video_name')

    # Loop through each row in the existing CSV file
    for row in existing_csv_reader:
        video_name = row[video_name_index].strip().lower()

        # Check if the video_name exists in the list
        if video_name in video_names_to_keep_lower:
            # Write the row to the new CSV file
            new_csv_writer.writerow(row)

print(f"Filtered CSV file has been created at {new_csv_path}.")


Filtered CSV file has been created at /content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/filtered_video_emotion_pairs.csv.




```
# This is formatted as code
```

#Concatenation


In [ ]:
import os

dir_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/final_concat_data/train/'

num_files = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])

print(num_files)

11053


In [ ]:
import os
import zipfile
from tqdm import tqdm

def zip_dir(dir_path, zip_path):
    """
    Zip all files in the given directory into a zip file.

    Args:
    - dir_path (str): Path to the directory containing the files to be zipped.
    - zip_path (str): Path to the zip file to be created.
    """
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        files = []
        for root, _, filenames in os.walk(dir_path):
            for filename in filenames:
                files.append(os.path.join(root, filename))
        with tqdm(total=len(files), desc='Zipping files', unit='file') as pbar:
            for file in files:
                zipf.write(file, os.path.relpath(file, dir_path))
                pbar.update(1)

# Directory to be zipped
dir_to_zip = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/final_concat_data/'

# Path for the zip file to be created
zip_file_path = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/concat.zip'

# Zip the directory
zip_dir(dir_to_zip, zip_file_path)

print("Directory zipped successfully!")


Zipping files: 100%|██████████| 13619/13619 [01:55<00:00, 118.05file/s]


Directory zipped successfully!


In [ ]:
import os
import shutil

# Source directory
source_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/final_concat_data/'

# Destination directory
destination_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/final_concat_data/train/'

# List all files in source directory
files = os.listdir(source_dir)

# Filter out only npy files
npy_files = [file for file in files if file.endswith('.npy')]

# Move each npy file to the destination directory
for file in npy_files:
    shutil.move(os.path.join(source_dir, file), os.path.join(destination_dir, file))

print("All npy files moved successfully!")


All npy files moved successfully!


In [ ]:
import os
import numpy as np
from tqdm import tqdm  # Import tqdm for the progress bar

# Define the source directories for video, text, and audio files
video_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/Video/train'
text_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/text/train'
audio_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/audio/train_mean'

# Define the target directory for concatenated npy files
output_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/final_concat_data/train'
os.makedirs(output_dir, exist_ok=True)

# Function to concatenate files with the same name
def concatenate_files(file_paths):
    # Load all files and concatenate along the specified axis
    concatenated_data = np.concatenate([np.load(file_path) for file_path in file_paths], axis=1)
    return concatenated_data

skipped = 0
# Iterate through each file in the 'video_v2' directory
for file_name in tqdm(os.listdir(video_dir), desc='Processing Files', unit='file'):
    video_path = os.path.join(video_dir, file_name)
    text_path = os.path.join(text_dir, file_name)
    audio_path = os.path.join(audio_dir, file_name)

    # Check if files with the same name exist in 'text' and 'audio' directories
    if os.path.isfile(text_path) and os.path.isfile(audio_path):
        # Define the output path for the concatenated npy file
        output_path = os.path.join(output_dir, file_name.replace('.npy', '_concatenated.npy'))

        # Check if the output file already exists
        if os.path.isfile(output_path):
          # tqdm.write(f"Skipping {file_name} - Output file already exists.")
          skipped += 1
          continue

        # Load, concatenate, and save the files
        # print(video_path)
        video_data = np.load(video_path)
        if video_data.shape == (768,):
          video_data = video_data.reshape(1, 768)
        text_data = np.load(text_path)
        audio_data = np.load(audio_path)

        # print("video_data", video_data.shape)
        # print("text_data", text_data.shape)
        # print("audio_data", audio_data.shape)

        concatenated_data = np.concatenate([video_data, text_data, audio_data], axis=1)
        np.save(output_path, concatenated_data)

        # tqdm.write(f"Concatenated data saved to {output_path}")

print("Processing completed.")
print('total skipped', skipped)


Processing Files: 100%|██████████| 11053/11053 [05:11<00:00, 35.52file/s]

Processing completed.
total skipped 7582


In [ ]:
import os
import numpy as np
from tqdm import tqdm  # Import tqdm for the progress bar

# Define the source directories for video, text, and audio files
video_dir = '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/video_v2'
text_dir = '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/text'
audio_dir = '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/audio_dev_embeddings_mean'

# Define the target directory for concatenated npy files
output_dir = '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/concatenated_data'
os.makedirs(output_dir, exist_ok=True)

# Function to concatenate files with the same name
def concatenate_files(file_paths):
    # Load all files and concatenate along the specified axis
    concatenated_data = np.concatenate([np.load(file_path) for file_path in file_paths], axis=1)
    return concatenated_data

skipped = 0
# Iterate through each file in the 'video_v2' directory
for file_name in tqdm(os.listdir(video_dir), desc='Processing Files', unit='file'):
    video_path = os.path.join(video_dir, file_name)
    text_path = os.path.join(text_dir, file_name)
    audio_path = os.path.join(audio_dir, file_name)

    # Check if files with the same name exist in 'text' and 'audio' directories
    if os.path.isfile(text_path) and os.path.isfile(audio_path):
        # Define the output path for the concatenated npy file
        output_path = os.path.join(output_dir, file_name.replace('.npy', '_concatenated.npy'))

        # Check if the output file already exists
        # if os.path.isfile(output_path):
        #     # tqdm.write(f"Skipping {file_name} - Output file already exists.")
        #     skipped += 1
        #     continue

        try:
            # Load, concatenate, and save the files
            video_data = np.load(video_path)
            text_data = np.load(text_path)
            audio_data = np.load(audio_path)

            concatenated_data = np.concatenate([video_data, text_data, audio_data], axis=1)
            np.save(output_path, concatenated_data)
        except Exception as e:
            tqdm.write(f"Skipping {file_name} - Error loading file: {str(e)}")
            skipped += 1

print("Processing completed.")
print('Total skipped:', skipped)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CSCI535 Project/Dataset/Processed/video_v2'

In [ ]:
import os

# Define the directory containing the concatenated data
output_dir = '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/concatenated_data'

# Count the number of files in the directory
num_files = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])

print(f"Number of files in '{output_dir}': {num_files}")


Number of files in '/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/Processed/concatenated_data': 1087


In [ ]:
import json

# Assuming your JSON data is stored in a string called json_data
# json_data = '''
# [
#     {
#         "conversation_ID": 1,
#         "conversation": [...],
#         "emotion-cause_pairs": [...]
#     },
#     {
#         "conversation_ID": 2,
#         "conversation": [...],
#         "emotion-cause_pairs": [...]
#     },
#     {
#         "conversation_ID": 3,
#         "conversation": [...],
#         "emotion-cause_pairs": [...]
#     }
# ]
# '''

with open('/content/drive/MyDrive/CSCI535/CSCI535 Project/Dataset/text/Subtask_2_train.json', 'r') as file:
    data = json.load(file)

# Load the JSON data
# data = json.loads(json_data)

# Initialize a variable to keep track of the maximum number of utterances
max_utterances = 0

# Iterate through each conversation in the data
for conversation in data:
    # Count the number of utterances in the current conversation
    utterance_count = len(conversation["conversation"])

    # Update max_utterances if the current conversation has more utterances
    if utterance_count > max_utterances:
        max_utterances = utterance_count

# Print the maximum number of utterances found
print(f"The maximum number of utterances in all the conversations is: {max_utterances}")


The maximum number of utterances in all the conversations is: 33
